In [2]:
import pandas as pd

from model2_final import main

#### Data Preprocessing and Feature Engineering

In [3]:
db_path = '../data/ORALMED_v10/Images'

df_mask = pd.read_csv('../data/ORALMED_v10/oralmedROI_v10.csv', sep=',')
df_info = pd.read_excel('../data/ORALMED_v10/oralmedDS_v10.xlsx', sheet_name='Paciente')
df_ds = pd.read_excel('../data/ORALMED_v10/oralmedDS_v10.xlsx', sheet_name='Imagen')
injury_mapping = pd.read_csv('../data/mapping.csv', sep='\t')


#### Model Training
First model training test - classification and segmentation at basic granularity level

In [4]:
#model_final
main(
    model_type = "custom_roi", # "base" or "custom_roi"
    images_dir = db_path,
    df_roi = df_mask,
    df_meta = df_ds,
    mapping_target_label = injury_mapping,
    output_dir = "./outcome_label",
    label_field = "outcome_label",   # "injury_label" or "outcome_label"
    train_ratio = 0.7,
    val_ratio = 0.15,
    test_ratio = 0.15,
    augmentation_type = 'light', # none = "base" or "light" or "strong"
    batch_size = 1,
    epochs = 25,
    lr = 0.005,
    score_thr = 0.00,
    mask_thr = 0.00,
    iou_thr = 0.50,
    dice_thr= 0.50,
    num_workers = 2,
    pretrained = True,
    use_amp = True,
    earlystop_metric = None, # "val"=None or "dice" or "iou" or "ap"
    segm_every = 1,
)

Classes: {1: 'benign', 2: 'malignant', 3: 'potentially malignant'}
Saved histogram to outcome_label/plots/case_label_distribution.png
Saved counts to outcome_label/plots/case_label_distribution.csv
Patient db size:  1738
train size:  1217 | val size:  261 | test size:  260
images -> train 3541 | val 781 | test 779
cuda
Mask-RCNN with CustomROI and weighted loss model selected...
Wrote val GT to outcome_label/coco_gt_val.json
Wrote test GT to outcome_label/coco_gt_test.json

=== Epoch 1/25 ===


Training step 1/25:   0%|          | 0/3541 [00:00<?, ?it/s]

train loss: 0.7045


Validation step 1/25:   0%|          | 0/781 [00:00<?, ?it/s]

validation loss: 0.7111
[simple_eval_limits] dets/img: 100 -> 20, score: 0.05 -> 0.0, min_size: [800] -> [512], max_size: 1333 -> 800


...

validation loss: 0.7427
[simple_eval_limits] dets/img: 100 -> 20, score: 0.05 -> 0.0, min_size: [800] -> [512], max_size: 1333 -> 800


Inference detections 2/25:   0%|          | 0/781 [00:00<?, ?it/s]

[simple_eval_limits] restored.
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Metrics saved at outcome_label/metrics_ckpt
LR now: 0.005000

=== Epoch 3/25 ===


Training step 3/25:   0%|          | 0/3541 [00:00<?, ?it/s]

KeyboardInterrupt: 

#### Inference - Test Set

#### Metrics & Visualization

In [ ]:
from error_report import save_detection_error_report
save_detection_error_report(
    gt_json=str("./outcome_label_output/coco_gt_val.json"),
    dt_json=str("./outcome_label_output/coco_dt_bbox_val.json"),
    out_csv=str("./outcome_label_output/val_errors_0p50.csv"),
    iou_thresh=0.50
)

'./outcome_label_output/val_errors_0p50.csv'

In [ ]:
from error_visualizer import visualize_errors_from_csv
counts = visualize_errors_from_csv(
    csv_path=str("./outcome_label_output/val_errors_0p50.csv"),
    images_root=db_path,
    out_dir=str("./outcome_label_output/val_error_viz"),
    max_images_per_type=20,
    resize_max=1600,
    style_scale=1.2,
    line_thickness=10,
    font_size=20,
    font_path="/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
)
print("Saved:", counts)

Saved: {'TP': 5, 'FP': 20, 'FN': 20}
